# Urban-Runoff-Pollutant-Model

This notebook implements a simplified urban runoff and pollutant load model based on SCS-CN hydrology and land-use–based pollutant export.

The workflow was originally developed as part of a graduate course assignment:

> **Hydrological Models: Sensitivity Analysis and Urban Runoff / Pollutant Estimation**

### Model capabilities

- Computes **area-weighted Curve Number (CN)** and crop coefficient (**k<sub>c</sub>**) at the basin scale  
- Uses **SCS-CN** method to estimate:
  - Potential maximum retention (**S**)
  - Initial abstraction (**I<sub>a</sub>**)
  - Daily evapotranspiration (**ET<sub>daily</sub>**)
  - Runoff depth (**P<sub>e</sub>**)
  - Infiltration (**F**)
- For each **subbasin**, it:
  - Computes runoff volume (m³ and liters)
  - Uses land-use–specific pollutant concentrations (TN, TP, BOD, TSS in mg/L)
  - Estimates total pollutant loads per subbasin (mg)
- Aggregates **total TN, TP, BOD, TSS** over all subbasins

### Input data

All inputs are provided as JSON files:

- `Scenario 1 (4)exam 1.json`  
  Contains precipitation, ET<sub>max</sub>, and a list of subbasins (area, land-use, hydrologic soil group, etc.)
- `land use and soil group .json`  
  Contains land-use classes, crop coefficient (**k<sub>c</sub>**) and Curve Numbers (CN) per soil group
- `pollutant concentrations Land uses (1).json`  
  Contains pollutant export concentrations (TN, TP, BOD, TSS) for each land-use code (LU_Code)

All files are expected to be located in:

```text
C:\Users\User\Urban-Runoff-Pollutant-Model\


In [18]:
import json
from pathlib import Path


# ==============================
# 1. Path settings (based on your system)
# ==============================

BASE_DIR = Path(r"C:\Users\User\Urban-Runoff-Pollutant-Model")

SCENARIO_FILE = BASE_DIR / "Scenario 1 (4)exam 1.json"
LANDUSE_FILE = BASE_DIR / "land use and soil group .json"
POLLUTANT_FILE = BASE_DIR / "pollutant concentrations Land uses (1).json"

OUTPUTS_DIR = BASE_DIR / "outputs"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)


# ==============================
# 2. Input class
# ==============================

class HydrologicInput:
    def __init__(self, scenario_path: Path, landuse_path: Path, pollutant_path: Path):
        self.scenario_path = scenario_path
        self.landuse_path = landuse_path
        self.pollutant_path = pollutant_path

        self.scenario_data = self._load_json_file(self.scenario_path)
        self.landuse_data = self._load_json_file(self.landuse_path)
        self.pollutant_data = self._load_json_file(self.pollutant_path)

    @staticmethod
    def _load_json_file(path: Path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"[ERROR] File not found: {path}")
            return None
        except json.JSONDecodeError:
            print(f"[ERROR] Invalid JSON format in file: {path}")
            return None


# ==============================
# 3. Output class (printing + saving)
# ==============================

class HydrologicOutput:
    @staticmethod
    def display_runoff_results(total_cn, final_kc, S, I_a, ET_daily, P_e, F):
        print("========== PART 1 – Runoff Calculation ==========\n")
        print(f"Total CN (area-weighted):        {total_cn:.3f}")
        print(f"Final k_c (area-weighted):       {final_kc:.3f}")
        print(f"S  (potential max loss) [mm]:    {S:.3f}")
        print(f"I_a (initial abstraction) [mm]:  {I_a:.3f}")
        print(f"ET_daily [mm]:                   {ET_daily:.3f}")
        print(f"P_e (runoff depth) [mm]:         {P_e:.3f}")
        print(f"F (infiltration) [mm]:           {F:.3f}")
        print("\n")

    @staticmethod
    def display_pollutant_results(subbasin_results):
        print("========== PART 2 – Pollutant Loads per Subbasin ==========\n")
        print(f"{'Subbasin':<10} {'LU_Code':<8} {'Total Pollutant (mg)':>20}")
        print("-" * 50)

        total_tn = 0.0
        total_tp = 0.0
        total_bod = 0.0
        total_tss = 0.0

        for res in subbasin_results:
            total_pollutant = (
                res["TN_Pollutant"]
                + res["TP_Pollutant"]
                + res["BOD_Pollutant"]
                + res["TSS_Pollutant"]
            )

            print(f"{res['Subbasin']:<10} {res['LU_Code']:<8} {total_pollutant:>20.2f} mg")
            print(
                f"    TN:  {res['TN_Pollutant']:.2f} mg  | "
                f"TP:  {res['TP_Pollutant']:.2f} mg  | "
                f"BOD: {res['BOD_Pollutant']:.2f} mg  | "
                f"TSS: {res['TSS_Pollutant']:.2f} mg"
            )
            print(
                f"    CN={res['CN']:.2f}, k_c={res['k_c']:.3f}, "
                f"S={res['S']:.2f} mm, I_a={res['I_a']:.2f} mm, "
                f"P_e={res['P_e']:.2f} mm, F={res['F']:.2f} mm"
            )
            print(
                f"    Runoff volume: {res['P_e_subbasin']:.3f} m³ "
                f"({res['P_e_subbasin_liters']:.1f} L)"
            )
            print("-" * 50)

            total_tn += res["TN_Pollutant"]
            total_tp += res["TP_Pollutant"]
            total_bod += res["BOD_Pollutant"]
            total_tss += res["TSS_Pollutant"]

        print("\n========== TOTAL POLLUTANTS (All Subbasins) ==========")
        print(f"Total TN :  {total_tn:.2f} mg")
        print(f"Total TP :  {total_tp:.2f} mg")
        print(f"Total BOD:  {total_bod:.2f} mg")
        print(f"Total TSS:  {total_tss:.2f} mg")
        print("======================================================\n")

    @staticmethod
    def save_to_json(path: Path, data: dict):
        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"[INFO] Results saved to: {path}")


# ==============================
# 4. Hydrologic model class
# ==============================

class HydrologicModel:
    def __init__(self, hydrologic_input: HydrologicInput):
        self.scenario_data = hydrologic_input.scenario_data
        self.landuse_data = hydrologic_input.landuse_data
        self.pollutant_data = hydrologic_input.pollutant_data

        if self.scenario_data is None:
            raise ValueError("Scenario JSON could not be loaded.")
        if self.landuse_data is None:
            raise ValueError("Landuse JSON could not be loaded.")
        if self.pollutant_data is None:
            raise ValueError("Pollutant JSON could not be loaded.")

    # ---------- Helper methods ----------

    def get_cn_and_kc(self, land_use: str, soil_group: str):
        """Return (CN, k_c) for given land use code and HSG."""
        for entry in self.landuse_data:
            if entry.get("Abbreviation") == land_use:
                cn = entry.get("CN", {}).get(soil_group)
                k_c = entry.get("k_c")
                return cn, k_c
        return None, None

    def get_lu_code(self, land_use: str):
        """Return land-use code (LU Code) for given land-use abbreviation."""
        for entry in self.landuse_data:
            if entry.get("Abbreviation") == land_use:
                return entry.get("LU Code")
        return None

    def get_pollutant_concentrations(self, lu_code: int | str):
        """Return (TN, TP, BOD, TSS) [mg/L] for a given LU_Code."""
        for pollutant in self.pollutant_data.get("land_use_codes", []):
            if pollutant.get("LU_Code") == lu_code:
                return (
                    pollutant.get("TN"),
                    pollutant.get("TP"),
                    pollutant.get("BOD"),
                    pollutant.get("TSS"),
                )
        return None, None, None, None

    # ---------- Part 1: Basin-average runoff ----------

    def calculate_total_runoff(self):
        subbasins = self.scenario_data.get("subbasins", [])
        total_area = sum(sb.get("area_m2", 0.0) for sb in subbasins)

        if total_area <= 0:
            raise ValueError("Total area is zero or negative.")

        weighted_cn_sum = 0.0
        weighted_kc_sum = 0.0

        for sb in subbasins:
            land_use = sb.get("land_use")
            soil_group = sb.get("hydrologic_soil_group")
            area = sb.get("area_m2", 0.0)

            cn, k_c = self.get_cn_and_kc(land_use, soil_group)
            if cn is None or k_c is None:
                print(f"[WARN] Missing CN or k_c for subbasin {sb.get('sub_basin')}")
                continue

            if not (0 < cn <= 100):
                print(f"[WARN] CN out of bounds in subbasin {sb.get('sub_basin')}: {cn}")
                continue

            weighted_cn_sum += cn * area
            weighted_kc_sum += k_c * area

        total_cn = weighted_cn_sum / total_area
        final_kc = weighted_kc_sum / total_area

        S = 25.4 * ((1000.0 / total_cn) - 10.0)         # [mm]
        I_a = 0.2 * S                                   # [mm]

        ET_max = self.scenario_data.get("ET_max")       # [mm/day]
        precipitation = self.scenario_data.get("precipitation")  # [mm]

        ET_daily = final_kc * ET_max

        if I_a > precipitation:
            P_e = 0.0
            print(f"[INFO] I_a ({I_a:.2f}) > P ({precipitation:.2f}) → P_e = 0")
        else:
            P_e = (precipitation - I_a) ** 2 / (precipitation - I_a + S)

        F = precipitation - P_e - I_a - ET_daily

        if F < 0:
            print("[INFO] Infiltration F < 0 → set to 0")
            F = 0.0

        return {
            "Total_CN": total_cn,
            "Final_k_c": final_kc,
            "S": S,
            "I_a": I_a,
            "ET_daily": ET_daily,
            "P_e": P_e,
            "F": F,
        }

    # ---------- Part 2: Pollutants per subbasin ----------

    def calculate_subbasin_pollutants(self):
        precipitation = self.scenario_data.get("precipitation")  # [mm]
        ET_max = self.scenario_data.get("ET_max")                # [mm/day]
        subbasins = self.scenario_data.get("subbasins", [])

        results = []

        for sb in subbasins:
            land_use = sb.get("land_use")
            soil_group = sb.get("hydrologic_soil_group")
            area = sb.get("area_m2", 0.0)
            sub_name = sb.get("sub_basin")

            cn, k_c = self.get_cn_and_kc(land_use, soil_group)
            if cn is None or k_c is None:
                print(f"[WARN] Missing CN/k_c for subbasin {sub_name}")
                continue

            if not (0 < cn <= 100):
                print(f"[WARN] CN out of bounds in subbasin {sub_name}: {cn}")
                continue

            S = 25.4 * ((1000.0 / cn) - 10.0)
            I_a = 0.2 * S
            ET_daily = k_c * ET_max

            if I_a > precipitation:
                P_e = 0.0
                print(f"[INFO] Subbasin {sub_name}: I_a > P → P_e = 0")
            else:
                P_e = (precipitation - I_a) ** 2 / (precipitation - I_a + S)

            # عمق رواناب به متر
            P_e_m = P_e / 1000.0
            # حجم رواناب (m³)
            P_e_subbasin = P_e_m * area
            # تبدیل به لیتر
            P_e_subbasin_liters = P_e_subbasin * 1000.0

            F = precipitation - P_e - I_a - ET_daily
            if F < 0:
                print(f"[INFO] Subbasin {sub_name}: F < 0 → set to 0")
                F = 0.0

            lu_code = self.get_lu_code(land_use)
            tn, tp, bod, tss = self.get_pollutant_concentrations(lu_code)

            tn_pollutant = P_e_subbasin_liters * tn if tn is not None else 0.0
            tp_pollutant = P_e_subbasin_liters * tp if tp is not None else 0.0
            bod_pollutant = P_e_subbasin_liters * bod if bod is not None else 0.0
            tss_pollutant = P_e_subbasin_liters * tss if tss is not None else 0.0

            results.append(
                {
                    "Subbasin": sub_name,
                    "CN": cn,
                    "k_c": k_c,
                    "S": S,
                    "I_a": I_a,
                    "ET_daily": ET_daily,
                    "P_e": P_e,
                    "P_e_subbasin": P_e_subbasin,
                    "P_e_subbasin_liters": P_e_subbasin_liters,
                    "F": F,
                    "LU_Code": lu_code,
                    "TN_Pollutant": tn_pollutant,
                    "TP_Pollutant": tp_pollutant,
                    "BOD_Pollutant": bod_pollutant,
                    "TSS_Pollutant": tss_pollutant,
                }
            )

        return results


# ==============================
# 5. Main – execution
# ==============================

def main():
    hydro_input = HydrologicInput(
        scenario_path=SCENARIO_FILE,
        landuse_path=LANDUSE_FILE,
        pollutant_path=POLLUTANT_FILE,
    )

    model = HydrologicModel(hydro_input)

    # Part 1: Runoff
    runoff_results = model.calculate_total_runoff()
    HydrologicOutput.display_runoff_results(
        total_cn=runoff_results["Total_CN"],
        final_kc=runoff_results["Final_k_c"],
        S=runoff_results["S"],
        I_a=runoff_results["I_a"],
        ET_daily=runoff_results["ET_daily"],
        P_e=runoff_results["P_e"],
        F=runoff_results["F"],
    )

    # Part 2: Pollutants
    subbasin_results = model.calculate_subbasin_pollutants()
    HydrologicOutput.display_pollutant_results(subbasin_results)

    # Save JSON output
    output_data = {
        "Runoff_Results": runoff_results,
        "Pollutant_Results": subbasin_results,
    }
    output_file = OUTPUTS_DIR / "output_results_exam1.json"
    HydrologicOutput.save_to_json(output_file, output_data)


# In Jupyter, just call::
main()


[INFO] I_a (31.49) > P (9.00) → P_e = 0
[INFO] Infiltration F < 0 → set to 0
========== PART 1 – Runoff Calculation ==========

Total CN (area-weighted):        61.732
Final k_c (area-weighted):       0.407
S  (potential max loss) [mm]:    157.456
I_a (initial abstraction) [mm]:  31.491
ET_daily [mm]:                   0.224
P_e (runoff depth) [mm]:         0.000
F (infiltration) [mm]:           0.000


[INFO] Subbasin 3: I_a > P → P_e = 0
[INFO] Subbasin 3: F < 0 → set to 0
========== PART 2 – Pollutant Loads per Subbasin ==========

Subbasin   LU_Code  Total Pollutant (mg)
--------------------------------------------------
1          23                 3819684.21 mg
    TN:  19687.61 mg  | TP:  3305.22 mg  | BOD: 3391442.72 mg  | TSS: 405248.66 mg
    CN=92.00, k_c=0.180, S=22.09 mm, I_a=4.42 mm, P_e=0.79 mm, F=3.70 mm
    Runoff volume: 2.874 m³ (2874.1 L)
--------------------------------------------------
2          24                 9186852.21 mg
    TN:  55606.05 mg  | TP:  8799